# TRIAL 2

## CNN model for dataset

## import library

In [ ]:
import torch
import torch.nn as nn
import torch.nn.init
from torch.utils.data import TensorDataset, DataLoader, Dataset

device = 'cpu'

import numpy as np

#### Visdom for visualization

In [ ]:
import visdom

vis = visdom.Visdom()
vis.close(env="main")

#### Define loss_tracker

In [ ]:
def loss_tracker(loss_plot, loss_value, num):
    vis.line(X=num,
             Y=loss_value,
             win = loss_plot,
             update='append'
             )

### Parameters

In [ ]:
learning_rate = 0.001
training_epochs = 10
batch_size = 64

### Load NumPy Dataset

In [ ]:
np_dataset = np.load('./data/np_dataset_split.npz')

In [ ]:
data = torch.from_numpy(np_dataset['data']).float()
targets = torch.from_numpy(np_dataset['targets']).long()
test_data = torch.from_numpy(np_dataset['test_data']).float()
test_targets = torch.from_numpy(np_dataset['test_targets']).long()

In [ ]:
class Custom_Dataset(Dataset):

    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        X = self.data[index]
        Y = self.targets[index]
        return X, Y

In [ ]:
train_dataset = Custom_Dataset(data=data, targets=targets)
test_dataset = Custom_Dataset(data=test_data, targets=test_targets)

In [ ]:
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          drop_last=True)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch_size,
                         shuffle=True,
                         drop_last=True)

In [ ]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(4,64,kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(3)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(64,256,kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(3)
        )
        self.fc1 = nn.Linear(256 * 28 * 42, 11, bias=True)

        torch.nn.init.xavier_uniform_(self.fc1.weight)
        
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        
        return out

In [ ]:
model = CNN().to(device)

In [ ]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate )

In [ ]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='loss_tracker', legend=['loss'], showlegend=True))

In [ ]:
total_batch = len(train_loader)

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in train_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost /total_batch
    print('[Epoch : {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))
print('Learning finished')

In [ ]:
with torch.no_grad():
    X_test = test_dataset.data.view(len(test_data), 4, 257, 382).float().to(device)
    Y_test = test_dataset.targets.to(device)

    prediction = model(X_test)
    
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy : ', accuracy.item())